In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import os

from unidecode import unidecode

### data preparation ###
root = os.path.expanduser('~/Documents/data/fake_real_news_dataset-master')
filepath = os.path.join(root,'fake_or_real_news.dat')
data = pd.read_csv(filepath)
# inpect data
data.head()
data.tail()
# inspect classes
print set(data.label)
print 'fake news n1 = ', sum(data['label']=='FAKE')
print 'real news n2 = ',sum(data['label']=='REAL')

# filter
import re
text_clean = []
for text in data['text']:
    text = re.sub(r'[^a-zA-Z]',' ',text)
    text = re.sub(r' +',' ',text)# remove superflous empty spaces
    text_clean.append(text.rstrip())
data['text_clean'] = text_clean

set(['REAL', 'FAKE'])
fake news n1 =  3164
real news n2 =  3171


In [2]:
ratio = .8
mask = np.random.rand(len(data)) <= ratio
data_train = data[mask]
data_test = data[~mask]


train_X = data_train['text_clean'].values
train_y = data_train['label'].values
test_X = data_test['text_clean'].values
test_y = data_test['label'].values

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics
from sklearn.linear_model import LogisticRegressionCV

vec = CountVectorizer()# instantiate vectorizer
train_feat = vec.fit_transform(train_X)
feat_names = vec.get_feature_names()

clf = LogisticRegressionCV()
clf.fit(train_feat,train_y)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [4]:
test_feat = vec.transform(test_X)
pred = clf.predict(test_feat)
confmat = metrics.confusion_matrix(test_y, pred)# horizontal: predicted label; vertical: true label
perf_acc = metrics.accuracy_score(test_y, pred)
perf_f1 = metrics.f1_score(test_y, pred, pos_label = 'REAL')

print metrics.classification_report(test_y, pred)

             precision    recall  f1-score   support

       FAKE       0.91      0.94      0.92       623
       REAL       0.93      0.91      0.92       599

avg / total       0.92      0.92      0.92      1222



In [7]:
import eli5
eli5.show_weights(clf, vec = vec, top=25)

Weight?,Feature
+0.374,candidates
+0.320,reform
+0.292,tuesday
+0.289,pushing
+0.286,but
+0.275,cruz
+0.273,gop
+0.268,consequences
+0.265,convention
+0.262,conservative


In [8]:
i = 2
text = data['text_clean'][i]
print data.label[i]
eli5.show_prediction(clf, text, vec=vec)

REAL
